# Aquí es la construcción del JSON con la información

In [1]:
import pdfplumber
import pandas as pd
import json 

In [ ]:
def parrafo(text,j,df=None):
    parrafo = text.split(" \n \n")
    new_parrafo = ' \n \n'.join(parrafo[:j])
    if df is None:
        return new_parrafo
    else :
        return new_parrafo + " \n \n" + df.to_string()

In [171]:
url = "./Reglamento-de-Transito-CDMX.pdf"

table_settings = {
    "vertical_strategy": "lines",
    "horizontal_strategy": "lines",
    "snap_x_tolerance": 10,
    #"intersection_x_tolerance": 15,
}

beta = {}


with pdfplumber.open(url) as pdf:
    alpha = [page.extract_text(keep_blank_chars=True) for page in pdf.pages]
    for page in pdf.pages:
        table = page.find_table()
        if table is not None:
            beta[page.page_number] = (page.crop(table.bbox),page.extract_table(table_settings),page.to_image())


for i, outer_list in enumerate(beta[10][1]):
    for j, inner_list in enumerate(outer_list):
        beta[10][1][i][j] = inner_list.replace("\n", " ")
        
for i, outer_list in enumerate(beta[11][1]):
    for j, inner_list in enumerate(outer_list):
        beta[11][1][i][j] = inner_list.replace("\n", " ")
        
for i, outer_list in enumerate(beta[67][1]):
    for j, inner_list in enumerate(outer_list):
        beta[67][1][i][j] = inner_list.replace("\n", " ")

for i, outer_list in enumerate(beta[68][1]):
    for j, inner_list in enumerate(outer_list):
        beta[68][1][i][j] = inner_list.replace("\n", " ")

for i, outer_list in enumerate(beta[132][1]):
    for j, inner_list in enumerate(outer_list):
        beta[132][1][i][j] = inner_list.replace("\n", " ")

df = pd.DataFrame(beta[10][1][1:],columns=beta[10][1][0])
df_2 = pd.DataFrame(beta[11][1][0:],columns=beta[10][1][0])
df_3 = pd.DataFrame(beta[67][1][1:],columns=beta[67][1][0])
df_4 = pd.DataFrame(beta[68][1][0:],columns=beta[67][1][0])
df_5 = pd.DataFrame(beta[132][1][1:],columns=beta[132][1][0])

df_12 = pd.concat([df,df_2],axis=0,ignore_index=True)
df_34 = pd.concat([df_3,df_4],axis=0,ignore_index=True)

alpha[9] = parrafo(alpha[9],-2,df_12)
alpha[10] = " \n \n".join(alpha[10].split(" \n \n")[2:])
alpha[66] = parrafo(alpha[66],-2,df_34)
alpha[67] = " \n \n".join(alpha[67].split(" \n \n")[2:])
alpha[131] = parrafo(alpha[131],-7,df_5)

# hasta la 74 (75 documento)
sample = alpha[:74]

prueba = " \n \n".join(sample)
prueba_2 = prueba.split("TÍTULO")

b = []
for i,celda in enumerate(prueba_2):
    for articulo in celda.split("Artículo"):
        a = {}
        a["Título"] = f"Título {i}"
        a["Artículo"] = "Artículo"+articulo
        a["Artículo"] = a["Artículo"].replace("\n \n \n \n  \n \n \n"," \n \n")
        a["Artículo"] = a["Artículo"].replace("\n \n \n  \n \n \n"," \n")
        
        b.append(a)

c = b[2::]

lista_quitar = [c[4],c[29],c[38],c[46],c[66],c[67],c[70]]

f = [x for x in c if x not in lista_quitar]

for indice in [5,12,18,20,23,32,38,49,52,58,68]:
    f[indice]['Artículo'] = parrafo(f[indice]['Artículo'],-2)

for indice in [44,60,69]:
    f[indice]['Artículo'] = f[indice]['Artículo'] + f[indice + 1]['Artículo']

ff = [x for x in f if x not in [f[45],f[61],f[70]]]


with open('documents.json', 'wt') as f_out:
    json.dump(ff, f_out, indent=2, ensure_ascii=False)


'''
Borrar lo de abajo:
f[5]['Artículo']
f[12]['Artículo']
(f[18]['Artículo']
f[20]['Artículo'])
f[23]['Artículo']
f[32]['Artículo']
f[38]['Artículo']
f[49]['Artículo']
f[52]['Artículo']
f[58]['Artículo']
f[68]['Artículo']

Juntar :
f[44]['Artículo'] - f[45]['Artículo']
f[60]['Artículo'] - f[61]['Artículo']
f[69]['Artículo'] - f[70]['Artículo']
'''


# Aqui es la construcción del RAG

In [13]:
from elasticsearch import Elasticsearch
from openai import OpenAI
import json

In [1]:
#preparing json
with open('documents.json', 'rt') as f_in:
    docs_raw = json.load(f_in)

In [14]:
#preparing elasticsearch
es_client = Elasticsearch('http://localhost:9200') 
es_client.info()

index_settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "Título": {"type": "keyword"},
            "Artículo": {"type": "text"} 
        }
    }
}

index_name = "transito-v1"

es_client.indices.delete(index=index_name, ignore_unavailable=True)
es_client.indices.create(index=index_name, body=index_settings)

for doc in docs_raw:
    try:
        es_client.index(index=index_name, document=doc)
    except Exception as e:
        print(e)

In [ ]:
#preparing openai
client = OpenAI()

In [21]:

def elastic_search(query_string):
    query_const={
        "match": {
            "Artículo":query_string
        }
    }
    response = es_client.search(index=index_name,query=query_const)

    results_docs = []

    for hit in response['hits']['hits']:
        results_docs.append(hit['_source'])

    return results_docs

def build_prompt(query, search_results):
    prompt_template = """
You're an assistant that knows the mexican traffic regulations, more specific the Ciudad de México traffic regulations. 
Answer the QUESTION based on the CONTEXT from the database, also the answer is need it to be in spanish.

QUESTION: {question}

CONTEXT: 
{context}
""".strip()

    context = ""
    
    for doc in search_results:
         context = context + f"Título: {doc['Título']}\nArtículo: {doc['Artículo']}\n\n"
    
    prompt = prompt_template.format(question=query, context=context).strip()
    return prompt

def llm(prompt):
    response = client.chat.completions.create(
        model='gpt-4o-mini',
        messages=[{"role": "user", "content": prompt}]
    )
    
    return response.choices[0].message.content

def rag(query):
    search_results = elastic_search(query)
    prompt = build_prompt(query, search_results)
    answer = llm(prompt)
    return answer

In [22]:
respuesta = rag("Qué dice el artículo 33")

In [24]:
print(respuesta)

El artículo 33 de las regulaciones de tránsito en la Ciudad de México establece que, aun cuando el conductor o alguna otra persona esté presente, los vehículos motorizados estacionados podrán ser inmovilizados por un agente autorizado en los siguientes casos:

I. Cuando los vehículos se encuentren estacionados en lugares prohibidos, de acuerdo con el artículo 30 en sus fracciones IV, VII, XIII y XVIII.

II. Para vehículos con placas foráneas estacionados en zonas con sistemas de cobro por estacionamiento en vía pública, en los siguientes casos:
   a) No se haya cubierto la cuota de estacionamiento en el momento de la revisión.
   b) Haya concluido el tiempo pagado.
   c) Si hay un comprobante de pago, éste no sea visible desde el exterior del vehículo, o el número de placas no coincida, o la fecha del comprobante sea distinta.
   d) El vehículo esté fuera de un cajón y/o zona marcada para el estacionamiento, o esté obstruyendo otro cajón o acceso a cochera, exceptuando aquellos vehícul